# 1. Instance Normalization

**提出的原因**：Batch Normalization 不适用于图像生成。因为在一个 mini-batch 中的图像有不同的风格，不能把这个 batch 里的数据都看作是同一类取标准化。

**思路**：逐个 instance 的 channel 计算均值和方差。也就是每个 feature map 计算一个均值和方差。

包括 InstanceNorm1d、InstanceNorm2d、InstanceNorm3d。

**以 InstanceNorm1d 为例介绍参数**：

`in = torch.nn.InstanceNorm1d(num_features, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)`

参数：
- num_features：一个样本的特征数，这个参数最重要
- eps：分母修正项
- momentum：指数加权平均估计当前的的均值和方差
- affine：是否需要 affine transform
- track_running_stats：True 为训练状态，此时均值和方差会根据每个 mini-batch 改变。False 为测试状态，此时均值和方差会固定

若输入数据的形状是 $B \times C \times 2Dfeature$，例如 (3, 3, 2, 2)，表示一个 mini-batch 有 3 个样本，每个样本有 3 个特征，每个特征的维度是 $2 \times 2 $。那么就会计算 $3 \times 3 $ 个均值和方差，分别对应每个样本的每个特征。


# 2. Group Normalization

**提出的原因**：在小 batch 的样本中，Batch Normalization 估计的值不准。

**思路**：数据不够，通道来凑。每个样本的特征分为几组，每组特征分别计算均值和方差。可以看作是 Layer Normalization 的基础上添加了特征分组。

**注意事项**：
- 不再有 running_mean 和 running_var
- $\gamma$ 和 $\beta$ 为逐通道的

**介绍参数**：

`gn = torch.nn.GroupNorm(num_groups, num_channels, eps=1e-05, affine=True)`

参数：
- num_groups：特征的分组数量
- num_channels：特征数，通道数。注意 num_channels 要可以整除 num_groups
- eps：分母修正项
- affine：是否需要 affine transform

若输入数据的形状是 $B \times C \times 2Dfeature$，例如 (2, 4, 3, 3)，表示一个 mini-batch 有 2 个样本，每个样本有 4 个特征，每个特征的维度是 $3 \times 3 $。num_groups 设置为 2，那么就会计算 $2 \times (4 \div 2) $ 个均值和方差，分别对应每个样本的每个特征。